<a href="https://colab.research.google.com/github/dipsankar-sinha/HO_ASR_Custom_Dataset/blob/main/ASR_For_HO_ipynb_1K_occ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install huggingface_hub
!pip install librosa
!pip install jiwer
!pip install --upgrade pyarrow
!pip install --upgrade huggingface_hub

In [2]:
!pip install transformers==4.42.0
!pip install torch==2.3.0+cu121 -f https://download.pytorch.org/whl/cu121/torch_stable.html
!pip install datasets==2.20.0
!pip install tokenizers==0.19.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 75.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
Looking in links: https://download.pytorch.org/whl/cu121/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install torchvision==0.18

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 49.9 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.19.0+cu121
    Uninstalling torchvision-0.19.0+cu121:
      Successfully uninstalled torchvision-0.19.0+cu121


In [4]:
!pip install torchaudio==2.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.1 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.4.0+cu121
    Uninstalling torchaudio-2.4.0+cu121:
      Successfully uninstalled torchaudio-2.4.0+cu121


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
from datasets import load_dataset

train = load_dataset("DipsankarSinha/HO_ASR_Corpus_1K_2024", split="train")
test = load_dataset("DipsankarSinha/HO_ASR_Corpus_1K_2024", split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

In [7]:
train

Dataset({
    features: ['path', 'text', 'audio', 'sampling_rate'],
    num_rows: 800
})

In [8]:
test

Dataset({
    features: ['path', 'text', 'audio', 'sampling_rate'],
    num_rows: 200
})

In [9]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(train.remove_columns(["path","audio"]), num_examples=10)

,text,sampling_rate
0,𑣞𑣈𑣓𑣉 𑣞𑣈𑣓𑣉 𑣒𑣉𑣌𑣈𑣅𑣁𑣍 𑣌𑣁𑣒𑣉 𑣁𑣋𑣃𑣛𑣂 𑣕𑣈 𑣙𑣃𑣛𑣂𑣓𑣊 𑣖𑣁𑣅𑣈𑣖 𑣁𑣚𑣕𑣁𑣓𑣁,16000
1,𑣜𑣁𑣗𑣁𑣋 𑣁𑣞𑣃 𑣓𑣁𑣖𑣁𑣌𑣆𑣙𑣈 𑣁𑣎 𑣙𑣉𑣓𑣁,16000
2,𑣁𑣍𑣁 𑣔𑣁𑣒𑣁 𑣗𑣃𑣒𑣈 𑣘𑣃𑣚𑣈𑣉 𑣅𑣁 𑣌𑣁𑣓𑣁 𑣌𑣁𑣜𑣌𑣁𑣑𑣈𑣓 𑣕𑣁𑣓 𑣜𑣈 𑣖𑣁𑣅𑣉𑣖 𑣉𑣚 𑣕𑣁𑣓𑣁,16000
3,𑣚𑣁𑣆 𑣙𑣃𑣅𑣈 𑣙𑣃𑣅𑣈 𑣙𑣃𑣗𑣉𑣕𑣁𑣓 𑣗𑣁𑣏𑣁𑣜 𑣁𑣜𑣈𑣑 𑣆𑣖𑣁𑣍 𑣗𑣈𑣓,16000
4,𑣁𑣓𑣁𑣎 𑣗𑣁𑣞𑣞𑣉 𑣙𑣃𑣗𑣁𑣉𑣌𑣁𑣓𑣁 𑣌𑣁𑣔 𑣔𑣂𑣋𑣁 𑣏𑣂𑣌𑣁𑣓𑣁 𑣙𑣃𑣗𑣉 𑣌𑣁𑣙𑣁 𑣔𑣂𑣅𑣈 𑣕𑣁𑣗𑣃,16000
5,𑣋𑣉𑣞𑣁𑣙𑣂 𑣁𑣍 𑣁𑣙𑣂 𑣏𑣁𑣑𑣃 𑣙𑣃𑣗𑣁𑣙𑣁𑣓𑣁 𑣋𑣉𑣁 𑣙𑣉𑣖𑣉𑣜𑣈 𑣋𑣃𑣑𑣁 𑣋𑣃𑣕𑣁𑣋𑣃𑣕𑣁𑣕𑣈 𑣏𑣂𑣓𑣁 𑣉𑣚𑣁𑣌𑣁𑣓𑣁,16000
6,𑣋𑣉𑣜𑣖𑣂 𑣌𑣁𑣒𑣁 𑣚𑣁𑣘𑣃𑣔𑣈 𑣌𑣁𑣓𑣁 𑣎𑣁𑣊 𑣑𑣁𑣌𑣒𑣛𑣈𑣍 𑣇𑣔𑣃𑣗𑣈𑣂 𑣙𑣈 𑣙𑣃𑣓𑣁𑣊 𑣏𑣃𑣈𑣚𑣈 𑣙𑣃𑣎𑣃𑣅𑣈,16000
7,𑣁𑣓𑣁𑣎 𑣗𑣁𑣞𑣈 𑣙𑣃𑣗𑣁𑣅𑣁𑣓𑣁 𑣌𑣁𑣙𑣁𑣔𑣉𑣁𑣓𑣔𑣉 𑣖𑣂𑣞𑣈 𑣌𑣁𑣍 𑣌𑣁𑣜𑣉𑣕𑣁𑣔𑣁 𑣓𑣈𑣓 𑣌𑣁𑣙𑣁𑣔𑣉 𑣕𑣈 𑣌𑣁𑣙𑣁 𑣙𑣃𑣗𑣁𑣅𑣁 𑣏𑣂,16000
8,𑣁𑣎𑣁 𑣁𑣖𑣉 𑣗𑣉𑣋𑣂𑣕𑣈 𑣖𑣃𑣒𑣁 𑣕𑣂 𑣌𑣁𑣒𑣁 𑣗𑣉𑣕𑣁𑣜𑣂𑣋𑣂𑣁 𑣁𑣎 𑣔𑣁𑣑𑣁 𑣞𑣁𑣗𑣁𑣌𑣈𑣕𑣈 𑣞𑣈𑣓𑣂𑣁,16000
9,𑣌𑣁𑣒𑣁 𑣔𑣉 𑣋𑣈𑣑𑣁 𑣌𑣓𑣁 𑣘𑣃𑣜𑣈𑣄 𑣖𑣅𑣉𑣖 𑣙𑣛𑣂𑣕𑣓𑣁,16000


In [11]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\\\u200d\dA-Za-z\’\‘\“\”\`\[\]\\/\\\\=]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]) + " "
    return batch

In [12]:
train = train.map(remove_special_characters)
test = test.map(remove_special_characters)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [13]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [14]:
vocab_train = train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train.column_names)
vocab_test = test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test.column_names)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [15]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'𑣇': 0,
 '𑣋': 1,
 '𑣂': 2,
 '𑣏': 3,
 '𑣀': 4,
 '𑣗': 5,
 '𑣁': 6,
 '𑣑': 7,
 '𑣅': 8,
 '𑣔': 9,
 '𑣊': 10,
 '𑣞': 11,
 '𑣜': 12,
 '𑣆': 13,
 '𑣎': 14,
 '𑣖': 15,
 '𑣈': 16,
 '𑣘': 17,
 '𑣝': 18,
 '𑣕': 19,
 '𑣓': 20,
 "'": 21,
 '𑣄': 22,
 '𑣍': 23,
 '𑣃': 24,
 '𑣉': 25,
 '𑣌': 26,
 '𑣙': 27,
 '𑣒': 28,
 '𑣟': 29,
 '𑣚': 30,
 '𑣛': 31,
 ' ': 32,
 '𑣐': 33}

In [16]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [17]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

36

In [18]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [19]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [20]:
repo_name = "HO_ASR_Custom_Dataset_1K_occ_2024"

In [21]:
tokenizer.push_to_hub(repo_name)

README.md:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DipsankarSinha/HO_ASR_Custom_Dataset_1K_occ_2024/commit/79b20f1667f1365a32b55b06b80f6405a86a4795', commit_message='Upload tokenizer', commit_description='', oid='79b20f1667f1365a32b55b06b80f6405a86a4795', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)


In [23]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


In [28]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train)-1)

print(train[rand_int]["text"])
ipd.Audio(data=train[rand_int]["audio"], autoplay=True, rate=16000)

𑣋𑣉𑣞𑣆 𑣁𑣍𑣁 𑣞𑣃𑣕𑣃𑣜𑣂 𑣗𑣁𑣜𑣞𑣁 𑣗𑣁𑣅𑣈𑣞𑣁 𑣗𑣁𑣜 𑣗𑣁𑣒𑣞𑣁 𑣙𑣃𑣗𑣁𑣉𑣕𑣁𑣓 𑣁𑣘𑣁𑣜𑣁 𑣚𑣈𑣞𑣂𑣞𑣂 


In [29]:
import numpy as np
import torchaudio

def preprocess(batch):
    np_arr = np.array(batch["audio"])
    np_flatten = np_arr.flatten()
    batch["audio"] = np.array(np_flatten)
    return batch

# Apply the preprocessing to the entire dataset
train = train.map(preprocess)
test = test.map(preprocess)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [30]:
train[7]

{'path': 'audio/1-1K/266.wav',
 'text': '𑣋𑣂𑣞𑣆 𑣁𑣍 𑣙𑣉𑣓 𑣓𑣁 𑣖𑣉𑣅𑣈 𑣗𑣁𑣞𑣈𑣁 𑣙𑣃𑣗𑣌𑣁𑣐 𑣗𑣁𑣜 𑣁𑣘𑣈 𑣁𑣓𑣔𑣃 𑣚𑣂𑣌𑣁 𑣙𑣃𑣗𑣁𑣌𑣁𑣓𑣁 𑣖𑣈𑣓𑣔𑣉 𑣖𑣈𑣑 𑣕𑣈 𑣖𑣈𑣑 𑣔 𑣁𑣚𑣁𑣉𑣕𑣁𑣓 ',
 'audio': [-2.1498851554254197e-14,
  -2.783531868541522e-13,
  -8.694041715934589e-14,
  -1.5828443698968908e-13,
  -1.2115761410629033e-13,
  -1.300060536654768e-12,
  8.1794673031238e-13,
  1.3541908479991482e-12,
  1.0432272450414115e-12,
  -1.6767856092329836e-13,
  4.1570913378308205e-12,
  4.083539062449404e-12,
  5.2793273225271165e-12,
  -9.881305843006949e-13,
  -4.797049801656683e-13,
  1.627087006794703e-11,
  2.5547175486195783e-11,
  2.9638187737379695e-11,
  2.994930345390223e-11,
  4.3120465531565344e-11,
  3.770969994598694e-11,
  6.063124828592503e-12,
  1.7453993111926636e-11,
  2.498280401996844e-11,
  5.057829155497018e-11,
  4.255501159788899e-11,
  1.9091218189659642e-11,
  6.469213359450166e-11,
  7.125443290956213e-11,
  3.756969041424085e-11,
  3.473109994045487e-11,
  1.281121458918122e-10,
  3.009645657692239e-11,
  7.61033042140

In [31]:
rand_int = random.randint(0, len(train)-1)

print("Target text:", train[rand_int]["text"])
print("Input array shape:", len(train[rand_int]["audio"]))
print("Sampling rate:", train[rand_int]["sampling_rate"])

Target text: 𑣁𑣓𑣁𑣎 𑣗𑣁𑣜𑣂𑣅 𑣕𑣈 𑣁𑣘𑣈 𑣏𑣁𑣓𑣔𑣃 𑣙𑣃𑣗𑣁𑣌𑣁𑣓𑣁 𑣖𑣂𑣞𑣈𑣉 𑣖𑣈𑣓 𑣁𑣙 𑣙𑣃𑣗𑣁𑣌𑣁𑣓𑣁 
Input array shape: 91952
Sampling rate: 16000


In [36]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train)-1)

print(train[rand_int]["text"])
ipd.Audio(data=train[rand_int]["audio"], autoplay=True, rate=16000)


𑣋𑣉𑣞𑣆 𑣁𑣎 𑣋𑣈𑣚𑣈𑣕𑣃𑣜𑣃𑣂 𑣗𑣉𑣜𑣞𑣁 𑣏𑣁𑣑𑣃𑣜𑣂𑣁 𑣇 𑣁𑣅𑣈𑣓𑣁 𑣚𑣁𑣕𑣁𑣜 𑣚𑣆 𑣗𑣉𑣋𑣂𑣕𑣈 𑣙𑣁𑣞𑣃𑣍 


In [37]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor.feature_extractor(raw_speech=audio, sampling_rate=16000).input_values[0] # Use feature_extractor to process audio data
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(text=batch["text"]).input_ids
    return batch


In [38]:
train

Dataset({
    features: ['path', 'text', 'audio', 'sampling_rate'],
    num_rows: 800
})

In [39]:
type(train['text'][0])

str

In [40]:
train = train.map(prepare_dataset, remove_columns=["path"])
test = test.map(prepare_dataset, remove_columns=["path"])

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [41]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch


In [42]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [43]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00


In [44]:
import evaluate

# Load the WER metric
wer_metric = evaluate.load("wer")

In [45]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [46]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)


config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [47]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2284: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [48]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=100,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [49]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=processor.feature_extractor,
)


In [50]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,13.097400,4.470391,1.000000
200,3.430500,3.032781,1.000000
300,3.004900,3.004130,1.000000
400,2.914900,2.861612,0.995699
500,2.814300,2.783839,1.000000
600,2.759400,2.746246,1.004839
700,2.703400,2.719675,0.981183
800,2.661900,2.722248,0.988172
900,2.611400,2.691331,0.983871
1000,2.559600,2.618711,1.004301


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of 

TrainOutput(global_step=1500, training_loss=3.366078847249349, metrics={'train_runtime': 5239.7839, 'train_samples_per_second': 4.58, 'train_steps_per_second': 0.286, 'total_flos': 5.658748861197153e+18, 'train_loss': 3.366078847249349, 'epoch': 30.0})

In [51]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/DipsankarSinha/HO_ASR_Custom_Dataset_1K_occ_2024/commit/6090af4f6b14859b8edc35b1b29c22c24f28377f', commit_message='End of training', commit_description='', oid='6090af4f6b14859b8edc35b1b29c22c24f28377f', pr_url=None, pr_revision=None, pr_num=None)

In [54]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("DipsankarSinha/HO_ASR_Custom_Dataset_1K_occ_2024").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("DipsankarSinha/HO_ASR_Custom_Dataset_1K_occ_2024")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [55]:
input_dict = processor(test[0]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits
pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [56]:
test_transcription = load_dataset("DipsankarSinha/HO_ASR_Corpus_1K_2024", split="test")

In [61]:
test

Dataset({
    features: ['text', 'audio', 'sampling_rate', 'input_values', 'input_length', 'labels'],
    num_rows: 200
})

In [62]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)

  return batch

results = test.map(map_to_result, remove_columns=['audio', 'sampling_rate', 'input_values', 'input_length', 'labels'])


Parameter 'function'=<function map_to_result at 0x7904f3d467a0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [64]:
print("Test WER:",wer_metric.compute(predictions=results["pred_str"], references=results["text"]))

Test WER: 1.0311827956989248


In [70]:
show_random_elements(results)

,text,pred_str
0,𑣎𑣁𑣙𑣁 𑣏𑣂𑣖𑣂𑣓𑣈𑣍 𑣎𑣁𑣖𑣈𑣕𑣁𑣜𑣂𑣙𑣁 𑣙𑣉𑣖𑣉𑣜𑣈 𑣌𑣁𑣙𑣁 𑣙𑣉𑣎𑣁𑣖𑣉𑣕𑣁𑣓𑣁,𑣋𑣁𑣞𑣁𑣁𑣁 𑣙𑣁 𑣁 𑣕𑣁𑣓 𑣗𑣁𑣌𑣁𑣕 𑣌𑣁 𑣙𑣁𑣕𑣁𑣓𑣁
1,𑣁𑣍'𑣄 𑣙𑣂𑣞𑣂𑣖𑣉𑣅𑣈𑣞𑣂𑣜𑣖𑣁 𑣇𑣖𑣈𑣜 𑣁𑣍'𑣄 𑣉𑣎𑣉𑣓 𑣖𑣂𑣞𑣉 𑣖𑣉𑣅𑣈𑣌𑣂𑣚𑣉,𑣋𑣉𑣞𑣁 𑣁𑣁𑣌𑣁 𑣌𑣁𑣕𑣁𑣓 𑣗𑣁𑣓𑣁 𑣌𑣁𑣕𑣁𑣓𑣁
2,𑣋𑣉𑣞𑣆 𑣁𑣓𑣁𑣁 𑣙𑣉𑣓𑣁 𑣌𑣃𑣅𑣃 𑣁𑣍𑣔𑣉 𑣙𑣁𑣞𑣃 𑣙𑣃𑣗𑣁𑣉𑣕𑣁𑣓𑣁 𑣜𑣈𑣑 𑣆𑣖𑣁𑣍 𑣗𑣈𑣓,𑣋𑣁𑣓𑣁 𑣁 𑣙𑣁𑣙𑣁𑣕𑣁𑣓𑣁 𑣗𑣁𑣌𑣁𑣙𑣁𑣕𑣁𑣓𑣁
3,𑣋𑣉𑣞𑣆 𑣁𑣓𑣁𑣎 𑣙𑣁𑣞𑣃 𑣕𑣈𑣓𑣈𑣍 𑣁𑣓𑣔𑣉 𑣌𑣃𑣙𑣃 𑣕𑣁𑣍 𑣁𑣍 𑣌𑣁𑣜𑣁𑣓𑣁 𑣌𑣓𑣁𑣍 𑣏𑣂,𑣋𑣉𑣞𑣆 𑣁𑣁𑣁 𑣌𑣁𑣕𑣁𑣓 𑣗𑣁𑣓𑣕𑣁 𑣌𑣁𑣕𑣁𑣓
4,𑣎𑣁𑣚𑣁𑣙𑣉𑣕𑣁𑣍 𑣗𑣁𑣏𑣁𑣜𑣁𑣌𑣁 𑣉𑣚𑣁𑣕𑣁𑣓𑣁 𑣁𑣑𑣁 𑣚𑣁𑣙𑣌 𑣖𑣂𑣞𑣈𑣖𑣂𑣞𑣈 𑣙𑣁𑣞𑣃𑣍,𑣋𑣉𑣞𑣆 𑣁 𑣌𑣁𑣕𑣁 𑣕𑣁 𑣌𑣁𑣁𑣕𑣁𑣓 𑣁𑣓𑣕𑣁 𑣌𑣁 𑣌𑣁𑣁𑣕𑣁𑣓𑣁
5,𑣆𑣁𑣙 𑣁𑣍𑣁 𑣙𑣁𑣚𑣘𑣈 𑣌𑣁𑣙𑣁𑣔𑣉 𑣁𑣓𑣜𑣈 𑣏𑣂𑣓𑣂𑣖 𑣒𑣁𑣌𑣁 𑣔𑣁𑣜𑣌𑣁𑣜𑣁,𑣋𑣉𑣞𑣁 𑣌𑣁𑣁𑣁 𑣕𑣁𑣓 𑣗𑣁𑣌 𑣁 𑣕𑣁𑣓𑣁
6,𑣋𑣉𑣞𑣆 𑣁𑣍𑣁 𑣙𑣉𑣓 𑣗𑣃𑣋𑣂𑣕𑣈𑣑 𑣌𑣃𑣅𑣃 𑣕𑣁𑣓𑣁 𑣌𑣃𑣅𑣃 𑣇𑣅𑣃 𑣕𑣈 𑣇𑣚𑣈 𑣜𑣂𑣋𑣉𑣅𑣈 𑣕𑣁𑣓𑣁,𑣋𑣉𑣞𑣆 𑣁𑣁𑣁 𑣙𑣁 𑣁𑣁 𑣁 𑣕𑣁𑣓𑣁 𑣗𑣁𑣌 𑣙𑣁 𑣁𑣌𑣁 𑣕𑣁𑣓
7,𑣗𑣉𑣙𑣁 𑣗𑣃𑣙𑣃𑣜 𑣗𑣂𑣙𑣃 𑣕𑣁𑣓𑣁 𑣎𑣃𑣌𑣁 𑣓𑣈𑣚𑣂𑣁𑣅 𑣏𑣂𑣗𑣈𑣓 𑣋𑣉𑣞𑣆,𑣋𑣉𑣞𑣆 𑣁 𑣌𑣁 𑣌𑣁 𑣕𑣁𑣓 𑣗𑣁𑣁 𑣌𑣁 𑣁 𑣕𑣁𑣓𑣁
8,𑣋𑣉𑣞𑣆 𑣁𑣍 𑣆𑣜𑣂𑣚 𑣏𑣁𑣓𑣔𑣃 𑣙𑣃𑣗𑣁𑣌𑣁𑣓𑣁 𑣋𑣉𑣅𑣁 𑣕𑣁𑣍 𑣕𑣂 𑣌𑣁𑣒𑣁 𑣖𑣉𑣙𑣁 𑣏𑣁𑣗𑣁 𑣌𑣁𑣓𑣁,𑣋𑣉𑣞𑣆 𑣁𑣙𑣁𑣙𑣁 𑣙𑣁𑣕𑣁𑣓 𑣁𑣓 𑣕𑣁 𑣌𑣁 𑣁 𑣙𑣁𑣕𑣁𑣓𑣁
9,𑣁𑣍'𑣄 𑣙𑣉𑣓𑣄 𑣚𑣃𑣕𑣃𑣜 𑣖𑣁𑣕𑣁 𑣌𑣁𑣓𑣁 𑣘𑣃𑣞𑣂𑣄 𑣉𑣄𑣚𑣕𑣁𑣓𑣁 𑣚𑣃𑣕𑣃𑣜 𑣑𑣁𑣌𑣕𑣉𑣜 𑣏𑣃𑣆𑣚𑣈 𑣙𑣃𑣎𑣃𑣄𑣅𑣈,𑣋𑣉𑣞𑣆 𑣁 𑣌𑣁𑣁𑣌𑣁𑣁 𑣌𑣁 𑣁𑣕𑣁𑣓𑣁 𑣗𑣁𑣓𑣕𑣁𑣁𑣁 𑣌𑣁𑣁 𑣕𑣁 𑣁 𑣕𑣁𑣓𑣁
